# Azure AutoML Local Compute Run 
## (Porto Seguro's Safe Driving Prediction)

Now let's use Azure Automated ML!

In this notebook you are going to use a **'Local Compute Run'** which is a better approach when getting started and trying small datasets because it needs less overhead on infrastructure preparation since it'll use your read-to-run Jupyter environment and compute from your machine (versus Azure ML remote runs which require to sping up VMs in the cloud for remote compute).

However, when targeting larger datasets and production trainings, it'll be better to use Azure ML Remote Compute since you will be able to parallelize runs/trainings and scale out on multiple VMs in the Azure ML training cluster. You can see that other approach in the 'automl-remote-compute-run-safe-driver-classifier.ipynb' notebook.

## Import Needed Packages

Import the packages needed for this solution notebook. The most widely used package for machine learning is [scikit-learn](https://scikit-learn.org/stable/), [pandas](https://pandas.pydata.org/docs/getting_started/index.html#getting-started), and [numpy](https://numpy.org/). 

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn import metrics

## Check Azure ML SDK version

In [ ]:
import azureml.core
print("This notebook was tested using version 1.6.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

##  Get Azure ML Workspace to use

In [ ]:
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

## Load data from file into regular Pandas DataFrame

In [ ]:
# Directly load from file into Pandas DataFrame
DATA_DIR = "../../data/"
data_df = pd.read_csv(os.path.join(DATA_DIR, 'porto_seguro_safe_driver_prediction_train.csv'))

print(data_df.shape)
data_df.head(5)

## Split Data into Train and Test Sets

Partitioning data into training, validation, and holdout/test sets allows you to develop highly accurate models that are relevant to data that you collect in the future, not just the data the model was trained on. 

In this notebook we holdout a test dataset to calculate the metrics with that set "not seen" by AutoML, after the AutoML process is finished.
Not taking into account the test dataset, AutoML will by default internally either use a validation dataset split from the train dataset or use cross-validation, depending on the size of the data (larger than 20k rows will use validation split), or as explicitely specified in the AutoMLConfig class (Validation-split vs. Cross-Validation).

In [ ]:
# Split in train/test datasets (Test=10%, Train=90%)

# Split in full sets, if not stratifying
train_df, test_df = train_test_split(data_df, test_size=0.1, random_state=0)

print(train_df.shape)
print(test_df.shape)

train_df.describe()

## Train with Azure AutoML automatically searching for the 'best model' (Best algorithms and best hyper-parameters)

### List and select primary metric to drive the AutoML classification problem

In [ ]:
from azureml.train import automl
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric

## Define AutoML Experiment settings

In [ ]:
import logging

# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
automl_settings = {
      "blacklist_models":['LogisticRegression', 'ExtremeRandomTrees', 'RandomForest'], 
      # "whitelist_models": ['LightGBM'],
      "validation_size": 0.1
      # "validation_data": validation_df,  # If you have an explicit validation set
      # "n_cross_validations": 5,
      # "experiment_exit_score": 0.7,
      # "max_cores_per_iteration": -1,
      "enable_voting_ensemble": 'False',
      "enable_stack_ensemble": 'False'
}

from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             primary_metric='AUC_weighted',                           
                             training_data= train_df,  # Pandas DataFrame
                             label_column_name="target",                                                    
                             enable_early_stopping= True,
                             iterations=15,                    
                             experiment_timeout_hours=1,
                             featurization="auto",
                             model_explainability=True,
                             **automl_settings
                             )

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

## Run Experiment with multiple child runs under the covers

In [ ]:
from azureml.core import Experiment

experiment_name = "SDK_local_porto_seguro_driver_pred"
print(experiment_name)

experiment = Experiment(workspace=ws, 
                        name=experiment_name)

import time
start_time = time.time()

run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s minutes needed for running the whole Remote AutoML Experiment ---' % ((time.time() - start_time)/60))

## Explore results with Widget

In [ ]:
# Explore the results of automatic training with a Jupyter widget: https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py
from azureml.widgets import RunDetails
RunDetails(run).show()

## Retrieve the 'Best' Model

In [ ]:
best_run, fitted_model = run.get_output()
print(best_run)
print('--------')
print(fitted_model)

## Make Predictions and calculate metrics

### Prep Test Data: Separate x_test set (feature columns) from y_test (label column) 

In [ ]:
import pandas as pd

x_test_df = test_df.copy()

if 'target' in x_test_df.columns:
    y_test_df = x_test_df.pop('target')

print(test_df.shape)
print(x_test_df.shape)
print(y_test_df.shape)


In [ ]:
y_test_df.describe()

### Make predictions in bulk

In [ ]:
# Try the best model making predictions with the test dataset
y_predictions = fitted_model.predict(x_test_df)

print('30 predictions: ')
print(y_predictions[:30])

### Get all the predictions' probabilities needed to calculate ROC AUC

In [ ]:
class_probabilities = fitted_model.predict_proba(x_test_df)

print(class_probabilities.shape)

print('Some class probabilities...: ')
print(class_probabilities[:3])
print('')
print('Probabilities for class 1:')
print(class_probabilities[:,1])
print('')
print('Probabilities for class 0:')
print(class_probabilities[:,0])

## Evaluate Model

Evaluating performance is an essential task in machine learning. In this case, because this is a classification problem, the data scientist elected to use an AUC - ROC Curve. When we need to check or visualize the performance of the multi - class classification problem, we use AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve. It is one of the most important evaluation metrics for checking any classification model’s performance.

<img src="https://www.researchgate.net/profile/Oxana_Trifonova/publication/276079439/figure/fig2/AS:614187332034565@1523445079168/An-example-of-ROC-curves-with-good-AUC-09-and-satisfactory-AUC-065-parameters.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 12px; width: 320px; height: 239px;" />

### Calculate the ROC AUC with probabilities vs. the Test Dataset

In [ ]:
print('ROC AUC *method 1*:')
fpr, tpr, thresholds = metrics.roc_curve(y_test_df, class_probabilities[:,1])
metrics.auc(fpr, tpr)


In [ ]:
from sklearn.metrics import roc_auc_score

print('ROC AUC *method 2*:')
print(roc_auc_score(y_test_df, class_probabilities[:,1]))

print('ROC AUC Weighted:')
print(roc_auc_score(y_test_df, class_probabilities[:,1], average='weighted'))


### Calculate the Accuracy with predictions vs. the Test Dataset

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
print(accuracy_score(y_test_df, y_predictions))


## Download model pickle file from the run

In [ ]:
# Download the model .pkl file to local (Using the 'run' object)
best_run.download_file('outputs/model.pkl')

### Load model in memory from .pkl file

In [ ]:
# Load the model into memory
import joblib
fitted_model = joblib.load('model.pkl')
print(fitted_model)